In [113]:
#step1.selenium 패키지와 time 모듈 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import pandas as pd

### DataFrame 행 넓이 최대 
pd.set_option('display.max_colwidth', -1)


<ipython-input-113-d81bf7df025c>:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [126]:
#step2.크롬드라이버로 url로 접속
url = 'https://www.teamblind.com/kr/'
driver = webdriver.Chrome(executable_path="/Users/USER/Desktop/datamining/chromedriver.exe")  ##크롬드라이버가 설치된 위치
driver.get(url)
time.sleep(3)


## 웹페이지 높이를 가져오기 위함
driver.maximize_window()


<ipython-input-126-20d3e369472b>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/Users/USER/Desktop/datamining/chromedriver.exe")  ##크롬드라이버가 설치된 위치


In [204]:
#step3.키워드 전처리 '·' 포함된 단어 모두 분할하여 키워드 리스트 재정리

category = ['디지털 드로잉', '드로잉', '공예', '요리 · 음료', '베이킹 · 디저트', '음악', '운동', '라이프', '사진 · 영상', '디자인', '개발 · 데이터', '직무교육', '글쓰기', '언어 · 외국어', '금융 · 재테크', '창업 · 부업', '성공 마인드', '아동 교육']

keyword_list=[]

for i in range(len(category)):
    keyword = category[i].split(' · ')
    keyword_list.extend(keyword)
    
print(len(keyword_list))
print(keyword_list)
print(keyword_list[7:])

25
['디지털 드로잉', '드로잉', '공예', '요리', '음료', '베이킹', '디저트', '음악', '운동', '라이프', '사진', '영상', '디자인', '개발', '데이터', '직무교육', '글쓰기', '언어', '외국어', '금융', '재테크', '창업', '부업', '성공 마인드', '아동 교육']
['음악', '운동', '라이프', '사진', '영상', '디자인', '개발', '데이터', '직무교육', '글쓰기', '언어', '외국어', '금융', '재테크', '창업', '부업', '성공 마인드', '아동 교육']


In [243]:
### 스크롤 다운 시간 설정 함수 

import datetime
    
    def doScrollDown(whileSeconds):
        start = datetime.datetime.now()
        end = start + datetime.timedelta(seconds=whileSeconds)
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(1)
            if datetime.datetime.now() > end:
                break

In [244]:
## 필요한 리스트 선언 
txt_list =[]      ### 내용
past_list =[]     ### 날짜
keywords =[]      ### 키워드

In [246]:
### keyword_list에 있는 키워드를 하나씩 가져와서 스롤링


for k in range(len(keyword_list)):

    query = keyword_list[k]
    search_box = driver.find_element_by_css_selector(".inp-srch")
    search_box.clear()
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(2)
    
    doScrollDown(10) ### 스크롤 다운 시간 (초 단위)
    


### 사용안함 ###

#    driver.execute_script("window.scrollTo(0,1000000)") ## 스크롤 길이 제한
#     SCROLL_PAUSE_SEC = 1

#     # 스크롤 높이 가져옴
#     last_height = driver.execute_script("return document.body.scrollHeight")

#     while True:
#         # 끝까지 스크롤 다운
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#         # 1초 대기
#         time.sleep(SCROLL_PAUSE_SEC)

#         # 스크롤 다운 후 스크롤 높이 다시 가져옴
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:
#             break
#         last_height = new_height
### 사용안함 ###    
    
    
    pre_txt = driver.find_elements_by_class_name('pre-txt')  ### 웹페이지 상에 내용 경로 지정
    pasts = driver.find_elements_by_class_name('past')       ### 웹페이지 상에 시간 경로 지정
    a = 0
    cnt = 0
    
    for i in pre_txt:
        txt = i.text
        txt_list.append(txt)
        keywords.append(query)
        past = pasts[a].text
        past_list.append(past)
        a= a+1
        cnt = cnt +1
        if(cnt == 500) :
            break
    
        ##for a in pasts:
            ##past = a.text
            
        ##past_list.append(past)
    
        
    

<ipython-input-246-f61b0d7b0911>:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_box = driver.find_element_by_css_selector(".inp-srch")
<ipython-input-246-f61b0d7b0911>:33: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  pre_txt = driver.find_elements_by_class_name('pre-txt')
<ipython-input-246-f61b0d7b0911>:34: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  pasts = driver.find_elements_by_class_name('past')


In [247]:
### 리스트를 딕셔너리형태로 만들어 DataFrame으로 변환

data = {"contents" : txt_list,
       "time" : past_list,
       "keyword" : keywords}  ## 출력된 데이터 양과 통일

In [248]:
### 리스트별 저장 데이터 개수 확인

print(len(txt_list), len(past_list), len(keywords))

5498 5498 5498


In [249]:
### DataFrame형태로 변환

df = pd.DataFrame(data)
df

,contents,time,keyword
0,라고 하기엔 3개 뿐이지만...😅 배우기 시작한지는 얼마 안되서 아직은 기초단계...,작성시간\n2020.08.27.,디지털 드로잉
1,아이패드 vs 타블렛 뭐가 나을까?? 클래스 101 인가 그거 들을만 할까 들어본 형 누나들??,작성시간\n2020.07.08.,디지털 드로잉
2,손으로 그리는 강의 추천하는 거 있나요?,작성시간\n2020.08.29.,디지털 드로잉
3,그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터디 식으로 크로키나 인체 연습 과제처럼 하는 그런 모임 없을까요~?,작성시간\n04.12,디지털 드로잉
4,요즘 반복되는 업무에 치이다보니 취미(좀 정적인것) 도전해보고 싶은데요 혹시 일산&합정&DMC 쪽에 취미로 해볼 수 있는 디지털드로잉 수업이나 강좌 있나요? 한번도 안해봐서 아직 장비는 없고 검색해봐두 전문학원이나 입시학원밖에 없어서요ㅠㅠ 이번에 D뮤지엄갔다가 디지털 드로잉에 꽂혀서 꼭 배워보고싶네요!! 취미로 하시는 분들 계시면 댓글 부탁드려요!^^,작성시간\n2019.06.12.,디지털 드로잉
...,...,...,...
5493,"생각의 흐름대로 편하게 적기 위해 간단체로 쓰니 양해바랍니다. ---------------------------------- 우연히 아래와 같이 너무 걱정되어 잠이 안온다는 하소연 글을 보게 되었다. 본문 이미지 나도 저런 신혼부부시절이 있었는데 벌써 시간이 많이 흘렀다. 세월 참 빠르다. 대학 시험 볼때, 결혼할 때, 아이 낳을 때가 어제 같은데 말이다. 위 글을 읽으면서 그 시간을 지나온 평범한 한 아빠로서 몇 생각이 들어 적어본다. 1. 아직 젊은데 너무 걱정할 것 없다. 언론에서 집값 상승한다고 너무 떠들기는 한",작성시간\n2020.07.02.,아동 교육
5494,"#오늘의뉴스 2020년 9월7일 1. 당정청, 7조원대 4차 추경 합의 → 전 국민 50%가량 지원 받을 듯. 특수고용직 등 고용 취약계층, 매출 감소 소상공인, 저소득층 긴급생계비, 통신비 지원 등 2500만명선.(국민) 2. 전체 범죄 감소하는데 사기 범죄는 늘어 → 2018년 전체 범죄는 4.8% 감소... 민생형 범죄인 온라인 거래, 부동산 사기등 사기범죄는 오히려 15.2% 늘어. <2019 범죄백서> 발표. 솜방이이 처벌도 원인. (세계) 3. ‘테스 형!’ → 나훈아 신곡 제목. 여기서 ‘테스 형’은 소크",작성시간\n2020.09.07.,아동 교육
5495,"생각의 흐름대로 편하게 적기 위해 간단체로 쓰니 양해바랍니다. ---------------------------------- 우연히 아래와 같이 너무 걱정되어 잠이 안온다는 하소연 글을 보게 되었다. 본문 이미지 나도 저런 신혼부부시절이 있었는데 벌써 시간이 많이 흘렀다. 세월 참 빠르다. 대학 시험 볼때, 결혼할 때, 아이 낳을 때가 어제 같은데 말이다. 위 글을 읽으면서 그 시간을 지나온 평범한 한 아빠로서 몇 생각이 들어 적어본다. 1. 아직 젊은데 너무 걱정할 것 없다. 언론에서 집값 상승한다고 너무 떠들기는 한",작성시간\n2020.07.02.,아동 교육
5496,"#오늘의뉴스 2020년 9월7일 1. 당정청, 7조원대 4차 추경 합의 → 전 국민 50%가량 지원 받을 듯. 특수고용직 등 고용 취약계층, 매출 감소 소상공인, 저소득층 긴급생계비, 통신비 지원 등 2500만명선.(국민) 2. 전체 범죄 감소하는데 사기 범죄는 늘어 → 2018년 전체 범죄는 4.8% 감소... 민생형 범죄인 온라인 거래, 부동산 사기등 사기범죄는 오히려 15.2% 늘어. <2019 범죄백서> 발표. 솜방이이 처벌도 원인. (세계) 3. ‘테스 형!’ → 나훈아 신곡 제목. 여기서 ‘테스 형’은 소크",작성시간\n2020.09.07.,아동 교육


In [250]:
df.shape

(5498, 3)

In [251]:
#### 키워드별 수집한 데이터 개수

freq = df['keyword'].value_counts() 
print(freq) 

사진         250
직무교육       250
금융         250
요리         250
영상         250
부업         250
음악         250
라이프        250
음료         250
외국어        250
베이킹        250
공예         250
개발         250
데이터        250
운동         250
창업         250
언어         230
디저트        230
디자인        210
드로잉        210
재테크        210
글쓰기        210
성공 마인드     125
아동 교육      62 
디지털 드로잉    11 
Name: keyword, dtype: int64


In [252]:
df.to_excel('blind_10s.xlsx')

In [253]:
df.to_csv('blind_10s.csv')